In [1]:
from fm_processor import preprocess_data

data_dir = "/Users/huylenguyen/digiLab/FM-digiLab-2025/data/raw/pipe01"
model_dir = "/Users/huylenguyen/digiLab/FM-digiLab-2025/data/processed/pipe01"

df_data = preprocess_data(data_dir, model_dir)

INFO:root:Reading raw data from /Users/huylenguyen/digiLab/FM-digiLab-2025/data/raw/pipe01 ...
INFO:root:Applying Wavelet denoising...
INFO:root:Applying baseline subtraction...
INFO:root:Applying SVD...
INFO:root:Saving preprocessed data to /Users/huylenguyen/digiLab/FM-digiLab-2025/data/processed/pipe01 ...
INFO:root:Data preprocessing finished.


In [78]:
import pandas as pd

df_train = pd.read_csv("/Users/huylenguyen/digiLab/FM-digiLab-2025/data/processed/pipe01/bss.csv", index_col=0)

df_train.head()

,filename_signal,filename_baseline,label,temperature,temperature_diff,0,1,2,3,4,...,48068,48069,48070,48071,48072,48073,48074,48075,48076,48077
0,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,0.161061,0.160822,0.160588,0.160369,0.160159,...,-0.044019,-0.044058,-0.044098,-0.044137,-0.044177,-0.044217,-0.044256,-0.044296,-0.044335,-0.044375
1,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,0.00,-0.025302,-0.025157,-0.025015,-0.024886,-0.024765,...,-0.187984,-0.188084,-0.188184,-0.188284,-0.188385,-0.188485,-0.188585,-0.188685,-0.188785,-0.188884
2,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,-0.075865,-0.075877,-0.075887,-0.075890,-0.075890,...,-0.654027,-0.654378,-0.654729,-0.655080,-0.655431,-0.655781,-0.656132,-0.656481,-0.656831,-0.657181
3,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.03,-0.226997,-0.227108,-0.227216,-0.227309,-0.227395,...,-0.740878,-0.741272,-0.741666,-0.742061,-0.742455,-0.742849,-0.743242,-0.743635,-0.744028,-0.744421
4,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,-0.052814,-0.052861,-0.052905,-0.052942,-0.052976,...,0.247200,0.247344,0.247489,0.247633,0.247778,0.247922,0.248066,0.248210,0.248354,0.248498


In [13]:
df_train_clean = df_train.loc[df_train.label == 0]
df_train_clean

,filename_signal,filename_baseline,label,temperature,temperature_diff,0,1,2,3,4,5
0,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,13.523828,-5.887021,77.482104,22.112726,-24.376199,12.760996
1,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,0.00,-27.063398,-9.791292,53.891984,32.738898,31.237525,-17.795159
2,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,-42.007306,-38.036218,95.081456,25.437930,-43.303168,44.766491
3,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.03,2.275988,-45.156123,123.182618,5.365477,-70.572260,-29.226575
4,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,15.52,-0.01,-40.765582,-16.111032,89.897011,41.478186,-2.841846,-1.650040
...,...,...,...,...,...,...,...,...,...,...,...
411,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,10.84,-4.68,-4684.378644,104.535779,-83.254121,0.265170,44.180869,-18.874776
412,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,10.84,0.18,39.182612,-4.145023,-70.702174,-83.146783,-53.941374,-23.010816
413,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,10.84,0.17,98.934223,-84.486032,-56.877389,-95.699191,-108.704326,-8.541270
414,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,/Users/huylenguyen/digiLab/FM-digiLab-2025/dat...,0,10.84,0.05,-54.793913,-41.065765,47.473792,29.510991,-19.186319,20.884311
